In [1]:
## Dependency libraries used for the SRG:
import numpy as np
import networkx as nx
import scipy as spy
import itertools
import copy
from scipy.signal import argrelextrema
from scipy.linalg import expm

In [27]:
def HighOrderLaplician(A, Type, Order):
    ## Input: 
    # A is the 1st-order adjacency matrix of the graph

    # Type is a str that determines which type of high order Laplician representation to use
    # Type='MOL': the function generates the multi-order Laplacian operator, which is 
    # Type='HOPL': the function generates the high-order path Laplacian operator proposed in our work

    # Order is a number that determines which order of interactions to analyze

    ## Output:
    # L is the high order Laplician representation, which is the Multiorder Laplacian operator or the high-order path Laplacian
    Num = A.shape[0]
    # Stores the vertices
    store = [0]* (Order+1)

    # Degree of the vertices
    # d = [deg for (_, deg) in G.degree()]
    d = list(np.sum(A,axis=0))

    Cliques = []
    # Function to check if the given set of vertices
    # in store array is a clique or not
    def is_clique(b) :
        # Run a loop for all the set of edges
        # for the select vertex
        for i in range(b) :
            for j in range(i + 1, b) :
                # If any edge is missing
                if (A[store[i]][store[j]] == 0) :
                    return False
        return True

    # Function to find all the cliques of size s
    def findCliques(i, l, s) :    
        # Check if any vertex from i+1 can be inserted as the l-th node in the simplex of size s
        for j in range( i + 1, Num) :
            # If the degree of the graph is sufficient
            if (d[j] >= s - 1) :
                # Add the vertex to store
                store[l] = j
                # If the graph is not a clique of size k
                # then it cannot be a clique
                # by adding another edge
                if (is_clique(l + 1)) :
                    # If the length of the clique is
                    # still less than the desired size
                    if (l < s-1) :
                        # Recursion to add vertices
                        findCliques(j, l + 1, s)
                    # Size is met
                    else :
                        Cliques.append(store[:s])
    findCliques(-1, 0, Order+1)

    if len(Cliques)>0:
        if Type=='MOL': # the multi-order Laplacian operator
            HO_A = np.zeros((Num, Num))
            for Simplex in Cliques:
                for [i, j] in itertools.combinations(Simplex,2):
                    HO_A[i,j] += 1
            HO_A += HO_A.T
            HO_D = np.zeros(Num)
            for Simplex in Cliques:
                for [i] in itertools.combinations(Simplex,1):
                    HO_D[i] += 1
            L = Order * np.diag(HO_D) - HO_A
        elif Type=='HOPL': # the high-order path Laplacian operator
            HO_B = np.zeros((Num, Num))
            for Simplex in Cliques:
                for [i, j] in itertools.combinations(Simplex,2):
                    HO_B[i,j] += np.math.factorial(Order-1)
            HO_B += HO_B.T
            HO_P = sum(HO_B)
            L = (np.diag(HO_P) - HO_B)/Order
    else:
        L=np.zeros((Num,Num))
    return L

def NetworkInfo(L):
    ## Input: 
    # L is the high order Laplician representation, which can be the Multiorder Laplacian operator or the high-order path Laplacian

    ## Output:
    # Sub_Ls the list of the sub-Laplician-matrices associated with all connected components from L
    # Sub_NodeIDs is the list of sub-node-indice associated with all connected components from L
    G=nx.from_numpy_array(np.abs(L)-np.diag(np.diag(L)))
    Sub_Ls=[L[list(SG),:][:,list(SG)] for SG in nx.connected_components(G)] 
    Sub_NodeIDs=[list(SG) for SG in nx.connected_components(G)]
    return Sub_Ls, Sub_NodeIDs

def NetworkUnion(Sub_Ls):
    L=np.ones((1,1))
    for SL in Sub_Ls:
        L=spy.linalg.block_diag(L,SL)
    L=L[1:,1:]
    return L

def SRG_Flow(G,q,p,L_Type,IterNum):
    A=nx.adjacency_matrix(G).toarray()       
    Lq=HighOrderLaplician(A, L_Type, Order=q)
    Lp=HighOrderLaplician(A, L_Type, Order=p)
    Lq_List,Lp_List,Gq_List,Gp_List,C_List,Tracked_Alignment=SRG_Function(Lq,Lp,q,IterNum)
    return  Lq_List,Lp_List,Gq_List,Gp_List,C_List,Tracked_Alignment

    
def SRG_Function(Lq,Lp,q,IterNum):
    ## Input: 
    # Lq is the initial guiding high order Laplician representation, which can be the Multiorder Laplacian operator or the high-order path Laplacian
    # Lp is the initial guided high order Laplician representation, which can be the Multiorder Laplacian operator or the high-order path Laplacian

    ## Output:
    # C_List is the list of specific heat vector calculated by the initial q - order Laplacian L_List[0] and a range of time scale
    # Lq_List is the list of the guiding q-order Laplacian over renormalization steps
    # Lp_List is the list of the guided p-order Laplacian over renormalization steps
    # Gq_List is the list of the guiding q-order network sketches over renormalization steps
    # Gp_List is the list of the guided p-order network sketches over renormalization steps
    # Tracked_Alignment is the indexes of the initial units aggregated into each macro-unit of all connected clusters after every iteration of the SRG
    
    Iter=1
    Gq=nx.from_numpy_array(np.abs(Lq)-np.diag(np.diag(Lq)))
    Gp=nx.from_numpy_array(np.abs(Lp)-np.diag(np.diag(Lp)))
    Lq_List=[Lq]
    Lp_List=[Lp]
    Gq_List=[Gq]
    Gp_List=[Gp]
    C_List=[]
    Init_Sub_Lqs,_=NetworkInfo(Lq)
    tau_Vec=np.zeros(len(Init_Sub_Lqs))
    for ID in range(len(Init_Sub_Lqs)):
        if np.size(Init_Sub_Lqs[ID],0)>1:
            tau,CVector=TauSelection(Init_Sub_Lqs[ID])
            tau=tau/((q+1)/2)
            tau_Vec[ID]=tau
            C_List.append(CVector)
    All_Alignment=[]
    while Iter<IterNum:
        ## step 1: initiate Laplacian operators and associated high-order network sketches in the k-th iteration
        Origin_Lq,Origin_Lp=Lq_List[Iter-1],Lp_List[Iter-1]
        Origin_Gq,Origin_Gp=Gq_List[Iter-1],Gp_List[Iter-1]
        Origin_Ap=nx.adjacency_matrix(Origin_Gp).toarray()
        New_Ap=copy.deepcopy(Origin_Ap)
        New_Lp=copy.deepcopy(Origin_Lp)
        Sub_Lqs, Sub_NodeIDs=NetworkInfo(Origin_Lq)
        Sub_Gqs=[Origin_Gq.subgraph(SNodeID).copy() for SNodeID in Sub_NodeIDs]
        NewSub_Lqs=[]
        NewSub_Gqs=[]
        New_Snodes_list=[]
        ClusterNodeAlignment=[]
        for SID in range(len(Sub_Lqs)):
            SLq=Sub_Lqs[SID]
            SGq=Sub_Gqs[SID]
            SNodeID = Sub_NodeIDs[SID]
            if np.size(SLq,0)==1:
                NodeAlignment=[]
                NewSub_Lqs.append(SLq)
                NewSub_Gqs.append(SGq)
                New_Snodes_list.extend(SNodeID)
                NodeAlignment.append([SNodeID[0],SNodeID])
            else:
                NodeAlignment=[]
                ## step 2: search targets to coarse grain in the real space
                Rho = expm(-tau*SLq)/np.trace(expm(-tau*SLq))
                diagonal_min = np.minimum(Rho.diagonal().reshape(-1, 1), Rho.diagonal())
                Rho_prime=Rho/diagonal_min
                Rho_prime=(Rho_prime+Rho_prime.T)/2
                Ref_adj=(Rho_prime>1).astype('int')
                # delete false edges not in subgraph of SLq
                Origin_SAq=nx.adjacency_matrix(SGq).toarray()
                Ref_mask=(Origin_SAq!=0).astype(int)
                Ref_adj=Ref_adj*Ref_mask
                RefG = nx.from_numpy_array(Ref_adj) # reference graph
                Clusters=[list(c) for c in list(nx.connected_components(RefG))]
                n_k=len(Clusters)
                ## step 3: implement a renormalization procedure for q-order and p-order network sketches in the real space
                # contracting SGq nodes
                New_SAq=copy.deepcopy(Origin_SAq)
                New_nodes=[Nodes[0] for Nodes in Clusters]
                for i in range(n_k):
                    iNodes=Clusters[i]
                    for j in range(i+1,n_k):
                        jNodes=Clusters[j]
                        new_weight=sum(Origin_SAq[iNodes,:][:,jNodes].reshape(-1))
                        New_SAq[iNodes[0],jNodes[0]]=new_weight
                        New_SAq[jNodes[0],iNodes[0]]=new_weight
                New_SAq=New_SAq[New_nodes,:][:,New_nodes]
                New_SGq=nx.from_numpy_array(New_SAq)
                NewSub_Gqs.append(New_SGq)
                # contracting Gp nodes
                New_Snodes=[SNodeID[Nodes[0]] for Nodes in Clusters]
                New_Snodes_list.extend(New_Snodes)
                Non_Nodes=[x for x in list(range(New_Ap.shape[0])) if x not in SNodeID]
                for i in range(n_k):
                    iSNodes = [SNodeID[node] for node in Clusters[i]]
                    New_Ap[iSNodes[0],Non_Nodes]=np.sum(Origin_Ap[iSNodes,:][:,Non_Nodes],axis=0) # merge edges from different connected components
                    for j in range(i+1,n_k):
                        jSNodes = [SNodeID[node] for node in Clusters[j]]
                        new_weight=sum(Origin_Ap[iSNodes,:][:,jSNodes].reshape(-1))  
                        New_Ap[iSNodes[0],jSNodes[0]]=new_weight
                        New_Ap[jSNodes[0],iSNodes[0]]=new_weight
                ## step 4: search modes to reduce in the moment space
                Evalqs, Evecqs=np.linalg.eig(SLq)
                tau=tau_Vec[SID]
                Evalq_idx=np.where(np.real(Evalqs)<1/tau)[0]
                m_k=len(Evalq_idx) if len(Evalq_idx)>=n_k else n_k
                ## step 5: calculate the re-scaled contributions of long-range eigenvectors
                SQq = np.zeros((SLq.shape[0],SLq.shape[1]))
                Evalq_idx=Evalqs.argsort()[:m_k]
                for ID in Evalq_idx:
                    Evecq = Evecqs[:,ID].reshape(-1,1)
                    SQq += np.real(Evalqs[ID]*np.matmul(Evecq,Evecq.T))

                SLp=Origin_Lp[SNodeID,:][:,SNodeID]
                Evalps, Evecps = np.linalg.eig(SLp)
                Evalp_idx=Evalps.argsort()[:m_k] 
                SQp = np.zeros((SLp.shape[0],SLp.shape[1]))
                for ID in Evalp_idx:
                    Evecp = Evecps[:,ID].reshape(-1,1)
                    SQp += np.real(Evalps[ID]*np.matmul(Evecp,Evecp.T))
                ## step 6: coarse grain q-order and p-order Laplacian in the momentum space 
                # calculate new SLq and SLp
                New_SLq = np.zeros((n_k,n_k))
                New_SLp = np.zeros((n_k,n_k))
                for i in range(n_k):
                    iNodes=Clusters[i]
                    iSNodes = [SNodeID[node] for node in Clusters[i]]
                    New_Lp[iSNodes[0],Non_Nodes]=np.sum(Origin_Lp[iSNodes,:][:,Non_Nodes],axis=0) # merge edges from different connected components
                    for j in range(i,n_k):
                        jNodes=Clusters[j]
                        new_weight=sum(SQq[iNodes,:][:,jNodes].reshape(-1))
                        new_weight0=sum(SQp[iNodes,:][:,jNodes].reshape(-1))
                        New_SLq[i,j]=New_SLq[j,i]=new_weight
                        New_SLp[i,j]=New_SLp[j,i]=new_weight0
                # remove false edges in SLq and SLp
                maskq=(New_SAq!=0).astype(int)
                New_SLq=New_SLq*maskq
                maskp=(New_Ap[New_Snodes,:][:,New_Snodes]).astype(int)
                New_SLp=New_SLp*maskp
                # ensure the sum of each row of q-order Laplacian operator is zero
                for i in range(n_k):
                    tmp=list(range(n_k))
                    tmp.remove(i)
                    New_SLq[i,i]=-np.sum(New_SLq[i,tmp])
                NewSub_Lqs.append(New_SLq)

                for i in range(n_k):
                    iNode1=New_Snodes[i]
                    for j in range(i+1,n_k):
                        jNode1=New_Snodes[j]
                        New_Lp[iNode1,jNode1]=New_SLp[i,j]
                        New_Lp[jNode1,iNode1]=New_SLp[j,i]
                # track node alignment
                for Nodes in Clusters:
                    SNodes = [SNodeID[node] for node in Nodes]
                    Node1 = SNodes[0]
                    NodeAlignment.append([Node1, SNodes])
            ClusterNodeAlignment.append(NodeAlignment)
        All_Alignment.append(ClusterNodeAlignment)

        New_Gq=nx.disjoint_union_all(NewSub_Gqs)
        New_Ap=New_Ap[New_Snodes_list,:][:,New_Snodes_list]
        New_Ap=np.maximum(New_Ap,New_Ap.T) # gaurantee the symmetry
        New_Gp=nx.from_numpy_array(New_Ap)
        New_Lp = New_Lp[New_Snodes_list,:][:,New_Snodes_list]
        New_Lp=np.minimum(New_Lp,New_Lp.T) # gaurantee the symmetry
        # ensure the sum of each row of p-order Laplacian operator is zero
        for i in range(New_Lp.shape[0]):
            tmp=list(range(New_Lp.shape[0]))
            tmp.remove(i)
            New_Lp[i,i]=-np.sum(New_Lp[i,tmp])
        New_Lq=NetworkUnion(NewSub_Lqs)

        Lq_List.append(New_Lq)
        Lp_List.append(New_Lp)
        Gq_List.append(New_Gq)
        Gp_List.append(New_Gp)
        Iter=Iter+1
    Tracked_Alignment=TrackingUnitID(All_Alignment,Lq.shape[0])
    return C_List,Lq_List,Lp_List,Gq_List,Gp_List,All_Alignment

def TauSelection(L):
    ## Input: 
    # L is the initial high order Laplician representation, which can be the Multiorder Laplacian operator or the high-order path Laplacian

    ## Output:
    # tau is the ideal constant that determines the time scale
    TauVec=np.logspace(-2, np.log10(50), 200)
    A=np.abs(L)-np.diag(np.diag(L))
    n_ks=[]
    MLambdaVector=np.zeros_like(TauVec)
    for ID in range(len(TauVec)):
        Poss_tau=TauVec[ID]
        MatrixExp=expm(-Poss_tau*L)
        Rho=MatrixExp/np.trace(MatrixExp)
        MLambdaVector[ID]=np.trace(L @ Rho)

        diagonal_min = np.minimum(Rho.diagonal().reshape(-1, 1), Rho.diagonal())
        Rho_prime=Rho/diagonal_min
        Rho_prime=(Rho_prime+Rho_prime.T)/2
        Ref_adj=(Rho_prime>1).astype('int')
        # delete false edges not in subgraph of SL
        Ref_mask=(A!=0).astype(int)
        Ref_adj=Ref_adj*Ref_mask
        RefG = nx.from_numpy_array(Ref_adj) # reference graph
        n_ks.append(nx.number_connected_components(RefG))
    CVector=-np.power(TauVec[1:],2)*np.diff(MLambdaVector)/np.diff(TauVec)
    CVector = CVector[np.where(~np.isnan(CVector))[0]]
    TauVec = TauVec[np.where(~np.isnan(CVector))[0]]

    potential_localmax_id = argrelextrema(CVector,np.greater)[0]
    true_localmax_id = potential_localmax_id[np.where(CVector[potential_localmax_id]>0.5*np.max(CVector))[0]]
    tau=TauVec[true_localmax_id[0]]

    dCVector=np.diff(CVector)/np.diff(TauVec)
    plateau_idx = np.where((np.abs(dCVector)<5e-2)&(TauVec[1:]>=tau))[0]
    plateau_idx = [i for i in plateau_idx if i+1 in plateau_idx] # find consecutive index as true plateau
    if len(plateau_idx)>50 and (CVector[plateau_idx]>0.1*np.max(CVector)).all():
        tau = TauVec[plateau_idx[0]]

    n_ks=np.array(n_ks)
    n_ks=n_ks[np.where(~np.isnan(CVector))[0]]
    dn_ks=np.abs(np.diff(n_ks)/np.diff(TauVec))
    bi_tau=TauVec[np.argmax(dn_ks)]
    tau=tau if tau>bi_tau else bi_tau
    print(['our method: tau is ',tau,'!!'])
    return tau,CVector

def TrackingUnitID(All_Alignment,UnitNum):
    # convert nodeID within an iteration (All_Alignment) to global nodeID (Tracked_Alignment)
    Tracked_Alignment=copy.deepcopy(All_Alignment)
    UnitIDVec=list(range(UnitNum))
    for IterID in range(len(All_Alignment)):
        if IterID>0:
            for ClusterID in range(len(All_Alignment[IterID])):
                for CoarseID in range(len(All_Alignment[IterID][ClusterID])):
                    NodesToTrack=All_Alignment[IterID][ClusterID][CoarseID][1]
                    for IDT in range(len(NodesToTrack)):
                        TrackedID=UnitIDVec[NodesToTrack[IDT]]
                        Tracked_Alignment[IterID][ClusterID][CoarseID][1][IDT]=TrackedID
                    Tracked_Alignment[IterID][ClusterID][CoarseID][0]=Tracked_Alignment[IterID][ClusterID][CoarseID][1][0]
        # delete Coarsed node
        NodetoDelete=[]
        for ClusterID in range(len(Tracked_Alignment[IterID])):
            for CoarseID in range(len(Tracked_Alignment[IterID][ClusterID])):
                Nodes=Tracked_Alignment[IterID][ClusterID][CoarseID][1][1:]
                NodetoDelete.extend(Nodes)

        for IDD in NodetoDelete:
            UnitIDVec.remove(IDD)

    return Tracked_Alignment

In [22]:
G=nx.random_graphs.barabasi_albert_graph(1000,3) # Generate a random BA network with 1000 units

In [34]:
# Multiorder Laplacian operator
Lq_List,Lp_List,Gq_List,Gp_List,C_List,Tracked_Alignment=SRG_Flow(G,q=1,p=1,L_Type='MOL',IterNum=5) # Run a SRG for 5 iterations, which renormalize the system on the 1-order based on the 1-order interactions

Lq_List,Lp_List,Gq_List,Gp_List,C_List,Tracked_Alignment=SRG_Flow(G,q=2,p=1,L_Type='MOL',IterNum=5) # Run a SRG for 5 iterations, which renormalize the system on the 1-order based on the 2-order interactions

Lq_List,Lp_List,Gq_List,Gp_List,C_List,Tracked_Alignment=SRG_Flow(G,q=3,p=1,L_Type='MOL',IterNum=5) # Run a SRG for 5 iterations, which renormalize the system on the 1-order based on the 3-order interactions

/tmp/ipykernel_1144124/835652801.py:100: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  A=nx.adjacency_matrix(G).toarray()


['our method: tau is ', 2.6086304875106583, '!!']
(5, 5)
(5, 5)
(5, 5)
(5, 5)
[0, [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 44, 45, 46, 47, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 

/tmp/ipykernel_1144124/835652801.py:141: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  Origin_Ap=nx.adjacency_matrix(Origin_Gp).toarray()
/tmp/ipykernel_1144124/835652801.py:169: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  Origin_SAq=nx.adjacency_matrix(SGq).toarray()


['our method: tau is ', 2.841764885639019, '!!']
['our method: tau is ', 12.710445049387584, '!!']
(164, 164)
(161, 161)
(161, 161)
(161, 161)
['our method: tau is ', 14.451872596765016, '!!']
(300, 300)
(299, 299)
(298, 298)
(298, 298)


In [21]:
# High-order path Laplacian
Lq_List,Lp_List,Gq_List,Gp_List,C_List,Tracked_Alignment=SRG_Flow(G,q=1,p=1,L_Type='HOPL',IterNum=5) # Run a SRG for 5 iterations, which renormalize the system on the 1-order based on the 1-order interactions

Lq_List,Lp_List,Gq_List,Gp_List,C_List,Tracked_Alignment=SRG_Flow(G,q=2,p=1,L_Type='HOPL',IterNum=5) # Run a SRG for 5 iterations, which renormalize the system on the 1-order based on the 2-order interactions

Lq_List,Lp_List,Gq_List,Gp_List,C_List,Tracked_Alignment=SRG_Flow(G,q=3,p=1,L_Type='HOPL',IterNum=5) # Run a SRG for 5 iterations, which renormalize the system on the 1-order based on the 3-order interactions

/tmp/ipykernel_1144124/2469768412.py:100: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  A=nx.adjacency_matrix(G).toarray()


['our method: tau is ', 4.359794641269904, '!!']
(36, 36)
(28, 28)


/tmp/ipykernel_1144124/2469768412.py:141: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  Origin_Ap=nx.adjacency_matrix(Origin_Gp).toarray()
/tmp/ipykernel_1144124/2469768412.py:169: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  Origin_SAq=nx.adjacency_matrix(SGq).toarray()


(28, 28)
(28, 28)
(300, 300)
(300, 300)
(300, 300)
(300, 300)
(300, 300)
(300, 300)
(300, 300)
(300, 300)
